In [1]:
# from src.dataset.process import process_data
# # data_file = "data/mini_sparkify_event_data.json"
# data_file = "data/mini_sparkify_event_data_sample.json"
# gather_col = "userId"
# sort_col = "ts"
# alias = "data"
# target_col = "page"
# exploded_col = "explode"
# emb_out_col = "embeddings"
# target_out_col = "targets"
# memory = "2g"
# process_data(
#     json_file=data_file,
#     gather_col=gather_col,
#     sort_col=sort_col,
#     alias=alias,
#     target_col=target_col,
#     exploded_col=exploded_col,
#     emb_out_col=emb_out_col,
#     target_out_col=target_out_col,
#     memory=memory
# )

In [2]:
from datasets import Dataset

In [3]:
ds_train_path = "data/train_dataset"
ds_test_path = "data/test_dataset"

In [4]:
train = Dataset.load_from_disk(ds_train_path)
test = Dataset.load_from_disk(ds_test_path)

In [5]:
sample = next(iter(train))

In [6]:
sample_test = next(iter(test))

In [7]:
from torch.utils.data import DataLoader

In [8]:
from src.preprocess.collate_fn import mat_collate_fn_dicts, mat_collate_fn

In [9]:
embeddings_key = "embeddings"
target_key = "targets"

In [10]:
dl = DataLoader(train, 
                batch_size=2, 
                collate_fn=lambda x: mat_collate_fn_dicts(x, embeddings_key, target_key)
                )

In [11]:
emb, pos_indices, targets, masks = next(iter(dl))

/run/media/john/Secondary/Documents/Cornell/Spring 2024/ORIE 4741 Learning From Data/Project/proj/lib64/python3.11/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  return _nested.nested_tensor(


In [12]:
from src.dataset.thin_wrapper import ThinWrapperDataset

In [13]:
wrap = ThinWrapperDataset(train, max_seq_len=100, embeddings_key=embeddings_key, targets_key=target_key)

In [14]:
dl_wrap = DataLoader(wrap,
                        batch_size=2,
                        collate_fn=mat_collate_fn
                        )

In [15]:
emb, pos_indices, targets, masks = next(iter(dl_wrap))

In [16]:
emb.shape

torch.Size([2, 100, 384])

In [21]:
test_target_dist_file = "data/test_target_dist.json"
train_target_dist_file = "data/train_target_dist.json"
unique_target_file = "data/unique_targets.json"
target_dist_file = "data/target_dist.json"

In [18]:
import json

In [22]:
with open(test_target_dist_file, "r") as f:
    test_target_dist = json.load(f)
with open(train_target_dist_file, "r") as f:
    train_target_dist = json.load(f)
with open(unique_target_file, "r") as f:
    unique_targets = json.load(f)
with open(target_dist_file, "r") as f:
    target_dist = json.load(f)

In [23]:
target_dist

{'Home': 136,
 'Logout': 37,
 'About': 13,
 'Settings': 17,
 'Login': 30,
 'Add to Playlist': 65,
 'Add Friend': 56,
 'NextSong': 2249,
 'Thumbs Up': 128,
 'Upgrade': 5,
 'Thumbs Down': 28,
 'Downgrade': 15,
 'Roll Advert': 52,
 'Help': 27,
 'Submit Upgrade': 2,
 'Save Settings': 3,
 'Submit Downgrade': 1}